In [1]:
import os
import cv2
import pandas as pd
import time
#from PIL import Image
import mediapipe as mp 
mpPose = mp.solutions.pose
pose = mpPose.Pose()
mpDraw = mp.solutions.drawing_utils # For drawing keypoints
points = mpPose.PoseLandmark # points will have all 33 landmarks like nose, eyes, ears etc.

In [2]:
data = list()
data.append('Image')
data.append('label')
rootdir = "/home/shikha/POSEESTIMATION/TRAIN"

In [3]:
for p in points: #p will iterate through nose, eyes, ears etc...
        x = str(p)[13:]
        data.append(x + "_x") #will append nose_x
        data.append(x + "_y") #will append nose_y
        data.append(x + "_z") #will append nose_z
        data.append(x + "_vis") #will append nose_vis
train_data = pd.DataFrame(columns = data)
num_images=0

In [4]:
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        try:
            temp=[]     
            imagename=os.path.basename(subdir)
            img = cv2.imread(rootdir + "/" +imagename+"/" +file)
            imageWidth, imageHeight = img.shape[:2]
            imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            temp.append(subdir)
            temp.append(imagename)
            results = pose.process(imgRGB)
            if results.pose_landmarks:
                    mpDraw.draw_landmarks(img, results.pose_landmarks, mpPose.POSE_CONNECTIONS) #draw landmarks on image
                    landmarks = results.pose_landmarks.landmark
                    for i,j in zip(points,landmarks):
                            temp = temp + [j.x, j.y, j.z, j.visibility]
            train_data.loc[num_images] = temp
            num_images += 1
        
        except ValueError as ve:
            continue 
train_data.to_csv("train_converted.csv", index=False)


In [5]:
from sklearn.svm import SVC
da = pd.read_csv("train_converted.csv")
X,Y= train_data.iloc[:,2:134],train_data['label']
svc = SVC(kernel = 'poly')
svc.fit(X,Y)


SVC(kernel='poly')

In [6]:
import pickle
Pkl_Filename = "coordinatemodel.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(svc, file)

In [7]:
test= list()
test.append('Image')
test.append('label')
testdir = "/home/shikha/POSEESTIMATION/TEST"

for p in points: #p will iterate through nose, eyes, ears etc...
        x = str(p)[13:]
        test.append(x + "_x")
        test.append(x + "_y")
        test.append(x + "_z")
        test.append(x + "_vis")
test_data = pd.DataFrame(columns = data)



In [8]:
test_images=0
for subdir, dirs, files in os.walk(testdir):
    for file in files:
        try:
            temp=[]   
            imagename_test=os.path.basename(subdir)
            img_test = cv2.imread(testdir + "/" +imagename_test+"/" +file)
            imageWidth, imageHeight = img_test.shape[:2]
            imgRGB = cv2.cvtColor(img_test, cv2.COLOR_BGR2RGB)
            temp.append(subdir)
            temp.append(imagename_test)
            results = pose.process(imgRGB)
            if results.pose_landmarks:
                    mpDraw.draw_landmarks(img_test, results.pose_landmarks, mpPose.POSE_CONNECTIONS) #draw landmarks on image
                    landmarks = results.pose_landmarks.landmark
                    for i,j in zip(points,landmarks):
                            temp = temp + [j.x, j.y, j.z, j.visibility]
            test_data.loc[test_images] = temp
            test_images += 1
        
        except ValueError as ve:
            continue 
test_data.to_csv("test_converted.csv", index=False)


In [9]:
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

clf = GaussianNB()

knc=KNeighborsClassifier()

rfc=RandomForestClassifier()

gbc=GradientBoostingClassifier()

In [10]:
S,T= test_data.iloc[:,2:134],test_data['label']
models = [svc,clf,knc,rfc,gbc]

In [13]:
for i in models:
    if(i==svc):
        print("\nSVC Classifier:\n")
    elif(i==clf):
        print("\nGaussian Naive Bayes:\n")
    elif(i==knc):
        print("\nKNeighbors Classifier:\n")
    elif(i==rfc):
        print("\nRandom Forest Classifier:\n")
    elif(i==gbc):
        print("\nGradient Boosting Classifier:\n")   
    modelfit = i.fit(S,T)
    pred=i.predict(S)
    acc=accuracy_score(T,pred)
    f1=f1_score(T, pred, average='micro')
    print ("Accuracy Score: {0:.4f}".format(acc))
    print("F1_score:",f1)


SVC Classifier:

Accuracy Score: 0.8769
F1_score: 0.8769050410316529

Gaussian Naive Bayes:

Accuracy Score: 0.5463
F1_score: 0.5463071512309496

KNeighbors Classifier:

Accuracy Score: 0.8054
F1_score: 0.8053927315357562

Random Forest Classifier:

Accuracy Score: 1.0000
F1_score: 1.0

Gradient Boosting Classifier:

Accuracy Score: 1.0000
F1_score: 1.0
